In [43]:
# TASK QUESTIONS:
# 2.1.1: What is the accuracy of this method?
#   26.49%
# 2.1.2: How has the accuracy changed now?
#   19.00%
# 2.1.3: Where/What is the bug?
#   It was a int overflow, it went from a 8 bit to a 32 bit (distance varable). 
#   Thus creating a overflow. This was fixed by converting the input arrays to 32 bits (.astype(int)).
# 2.2.2: What is the accuracy with best k?
#   Depends on permutation, see code prints

import numpy as np
from matplotlib import pyplot as plt
import random

In [44]:
from keras.datasets import mnist
(Xtr, Ltr), (test_d, L_test) = mnist.load_data()

In [45]:
num_sample=500
Tr_set=Xtr[:num_sample,:,:]
Ltr_set=Ltr[:num_sample]

# Bugged: Tr_set=Tr_set.reshape(num_sample,Tr_set.shape[1]*Tr_set.shape[2])

# FIXED VIA .astype(int) on training set in this block
# When printing type(..) on operations in the 1-nn code,
# note that the training set types = int8 and the distance types = int32
# presuming some sort of overflow or other arithmetic error

Tr_set=Tr_set.reshape(num_sample,Tr_set.shape[1]*Tr_set.shape[2])
Tr_set=Tr_set.astype(int)

In [46]:
# 2.1.1
def predict_1NN_L1(X):
    num_test = X.shape[0]
    Lpred=np.zeros(num_test, dtype = Ltr_set.dtype)
    
    for i in range(num_test):
        distances = np.sum(np.abs(Tr_set-X[i,:]),axis=1)
        min_index = np.argmin(distances)
        Lpred[i] = Ltr_set[min_index]
        
    return Lpred

In [47]:
# 2.1.2
def predict_1NN_L2(X):
    num_test = X.shape[0]
    Lpred=np.zeros(num_test, dtype = Ltr_set.dtype)
    for i in range(num_test):
        distances = np.sqrt(np.sum((np.abs(Tr_set-X[i,:])**2), axis=1))       
        min_index = np.argmin(distances)
        Lpred[i] = Ltr_set[min_index]
    return Lpred

In [48]:
# 2.1.4
# find the most common element in a list
def mostCommonElement(l):
    vals = {}
    curr_max = curr_val = 0
    for e in l:
        if e not in vals: vals[e] = 0
        vals[e] += 1
        if vals[e] > curr_max:
            curr_max = vals[e]
            curr_val = e
    return curr_val       

# k-NN using L2 norm
def predict_kNN_L2(k, test_d, train_d, train_l):
    num_test = test_d.shape[0]
    Lpred = np.zeros(num_test, dtype = train_l.dtype)
    for i in range(num_test):
        distances = np.sqrt(np.sum((np.abs(train_d-test_d[i,:])**2), axis=1))

        # vote
        best_guesses = []
        for j in range(0, k):
            min_index = np.argmin(distances)
            best_guesses.append(Ltr_set[min_index])
            distances = np.delete(distances, min_index) # already marked; remove from possible neighbours

        Lpred[i] = mostCommonElement(best_guesses)    
    return Lpred

In [49]:
# 2.2 helper functions
def permuteKnnTestSet(data, labels):
    order = np.arange(len(data))
    np.random.shuffle(order)
    return data[order], labels[order]

In [57]:
# 2.2
# return value and index of the element in the list with highest value
def highestElement(l):
        max_v, max_i = l[0], 0
        for i in range(0, len(l)):
            if l[i] > max_v:
                max_v = l[i]
                max_i = i
        return max_v, max_i

# perform 3-fold cross-validation to find best k value in range (0,kmax)
# returns best k value found
def cv_3fold(test_d, test_l, kmax=10):
    no_folds = 3
    best_k = []
    for i in range(0, no_folds):
        test_d, test_l = permuteKnnTestSet(test_d, test_l)
        folds = np.array_split(test_d, no_folds) 
        folds_labels = np.array_split(test_l, no_folds) # labels

        # divide data into folds
        test_set, train_set = folds.pop(i), np.concatenate((folds[0], folds[1]))
        test_labels, train_labels = folds_labels.pop(i), np.concatenate((folds[0], folds[1]))

        # find best k for this fold
        accuracies = []
        for k in range(0, kmax):
            accuracies.append(np.mean(predict_kNN_L2(k, test_set, train_set, train_labels) == test_labels))
        v, k = highestElement(accuracies)
        #print(v, k)
        best_k.append(k)
    return mostCommonElement(best_k)
    
best_k = cv_3fold(Tr_set, Ltr_set)
print(best_k)

# Check acc for best k found
Test_images = test_d.reshape(test_d.shape[0],test_d.shape[1]* test_d.shape[2])
print(np.mean(predict_kNN_L2(best_k, Test_images, Tr_set, Ltr_set) == L_test))

1
0.8294


In [51]:
Test_images=test_d.reshape(test_d.shape[0],test_d.shape[1]* test_d.shape[2])
Lpred2 = predict_1NN_L2(Test_images)
print("Accuracy:", np.mean(Lpred2==L_test))

Accuracy: 0.8294
